## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tazovskiy,RU,67.4667,78.7000,1.62,97,100,8.77,overcast clouds
1,1,Leningradskiy,RU,69.3833,178.4167,-5.91,97,100,11.79,overcast clouds
2,2,Tasiilaq,GL,65.6145,-37.6368,23.09,90,100,9.46,light snow
3,3,Chernyshevskiy,RU,63.0128,112.4714,20.05,95,100,8.59,overcast clouds
4,4,Mosetse,BW,-20.7500,26.6500,83.10,28,46,5.44,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Mosetse,BW,-20.7500,26.6500,83.10,28,46,5.44,scattered clouds
6,6,Sakakah,SA,29.9697,40.2064,83.77,18,76,6.11,broken clouds
8,8,San Patricio,US,28.0170,-97.5169,76.15,16,0,16.84,clear sky
11,11,Vaini,TO,-21.2000,-175.2000,77.16,94,75,2.30,broken clouds
12,12,Rikitea,PF,-23.1203,-134.9692,79.68,63,7,14.14,clear sky
13,13,Rocha,UY,-34.4833,-54.3333,81.68,34,100,4.94,overcast clouds
15,15,Mahebourg,MU,-20.4081,57.7000,79.09,83,40,11.50,scattered clouds
18,18,Kanniyakumari,IN,8.0833,77.5667,83.66,74,24,6.40,few clouds
22,22,Butaritari,KI,3.0707,172.7902,80.69,81,100,16.84,light rain
27,27,Bontang,ID,0.1333,117.5000,76.62,90,96,7.40,overcast clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                225
City                   225
Country                222
Lat                    225
Lng                    225
Max Temp               225
Humidity               225
Cloudiness             225
Wind Speed             225
Current Description    225
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Mosetse,BW,83.10,scattered clouds,-20.7500,26.6500,
6,Sakakah,SA,83.77,broken clouds,29.9697,40.2064,
8,San Patricio,US,76.15,clear sky,28.0170,-97.5169,
11,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,
12,Rikitea,PF,79.68,clear sky,-23.1203,-134.9692,
13,Rocha,UY,81.68,overcast clouds,-34.4833,-54.3333,
15,Mahebourg,MU,79.09,scattered clouds,-20.4081,57.7000,
18,Kanniyakumari,IN,83.66,few clouds,8.0833,77.5667,
22,Butaritari,KI,80.69,light rain,3.0707,172.7902,
27,Bontang,ID,76.62,overcast clouds,0.1333,117.5000,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
# 6c. Get latitude and longitude from DataFrame
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
   
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
   

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"]!= '']
#clean_hotel_df = hotel_df.dropna()

In [11]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Sakakah,SA,83.77,broken clouds,29.9697,40.2064,Fakhamat Al Orjoana Apartment
8,San Patricio,US,76.15,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
11,Vaini,TO,77.16,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
12,Rikitea,PF,79.68,clear sky,-23.1203,-134.9692,People ThankYou
13,Rocha,UY,81.68,overcast clouds,-34.4833,-54.3333,Hotel Arrarte
15,Mahebourg,MU,79.09,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
18,Kanniyakumari,IN,83.66,few clouds,8.0833,77.5667,Sparsa Kanyakumari
22,Butaritari,KI,80.69,light rain,3.0707,172.7902,Isles Sunset Lodge
27,Bontang,ID,76.62,overcast clouds,0.1333,117.5000,KOPKAR Hotel Bintang Sintuk
30,Komsomolskiy,UZ,77.72,overcast clouds,40.4272,71.7189,Ikathouse


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))